In [1]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt


In [2]:
df=pd.read_csv("spam.csv",encoding="ISO-8859-1")

In [3]:
df=df[['v1','v2']]
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
df=df.rename(columns={'v1':"spam",'v2':'text'})
df

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
df.spam=df.spam.apply(lambda s: True if s=='spam' else False)
df

,spam,text
0,False,"Go until jurong point, crazy.. Available only ..."
1,False,Ok lar... Joking wif u oni...
2,True,Free entry in 2 a wkly comp to win FA Cup fina...
3,False,U dun say so early hor... U c already then say...
4,False,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,True,This is the 2nd time we have tried 2 contact u...
5568,False,Will Ì_ b going to esplanade fr home?
5569,False,"Pity, * was in mood for that. So...any other s..."
5570,False,The guy did some bitching but I acted like i'd...


In [6]:
df.text=df.text.str.lower().str.translate(str.maketrans('', '', string.punctuation))
df

,spam,text
0,False,go until jurong point crazy available only in ...
1,False,ok lar joking wif u oni
2,True,free entry in 2 a wkly comp to win fa cup fina...
3,False,u dun say so early hor u c already then say
4,False,nah i dont think he goes to usf he lives aroun...
...,...,...
5567,True,this is the 2nd time we have tried 2 contact u...
5568,False,will ì b going to esplanade fr home
5569,False,pity was in mood for that soany other suggest...
5570,False,the guy did some bitching but i acted like id ...


In [7]:
df=df.sample(frac=1)
df

,spam,text
2664,False,he remains a bro amongst bros
5568,False,will ì b going to esplanade fr home
4966,True,you can donate å£250 to unicefs asian tsunami ...
4843,True,you have won as a valued vodafone customer our...
3723,True,hi there 2nights ur lucky night uve been invit...
...,...,...
956,False,love isnt a decision its a feeling if we could...
5442,False,i thought id get him a watch just cos thats th...
4991,False,im hungry buy smth home
2757,False,yo we are watching a movie on netflix


In [8]:
for i in df.loc[df.spam[df.spam==True].iloc[:5].index, 'text']:
    print(i)
    print("----------")


you can donate å£250 to unicefs asian tsunami disaster support fund by texting donate to 864233 å£250 will be added to your next bill
----------
you have won as a valued vodafone customer our computer has picked you to win a å£150 prize to collect is easy just call 09061743386 
----------
hi there 2nights ur lucky night uve been invited 2 xchat the uks wildest chat txt chat to 86688 now 150pmsgrcvdhgsuite3422landsroww1j6hl ldn 18yrs
----------
win a year supply of cds 4 a store of ur choice worth å£500  enter our å£100 weekly draw txt music to 87066 tscs wwwldewcomsubs161win150ppmx3
----------
the current leading bid is 151 to pause this auction send out customer care 08718726270
----------


In [9]:
for i in df.loc[df.spam[df.spam==False].iloc[:5].index, 'text']:
    print(i)
    print("----------")

he remains a bro amongst bros
----------
will ì b going to esplanade fr home
----------
ok c ya
----------
ìï come lt 25 n pass to me lar
----------
will do was exhausted on train this morning too much wine and pie you sleep well too
----------


In [10]:
training_df=df.iloc[:int(len(df)*0.7)]
testing_df=df.iloc[int(len(df)*0.7):]

In [11]:
fraction_spam=training_df.spam.mean()

In [12]:
train_spam_words=''.join(training_df[training_df.spam==True].text).split(' ')

In [13]:
train_nonspam_words=''.join(training_df[training_df.spam==False].text).split(' ')

In [14]:
common_words=set(train_spam_words).intersection(set(train_nonspam_words))

In [15]:
train_spam_bow=dict()
for w in common_words:
    train_spam_bow[w]=train_spam_words.count(w)/len(train_spam_words)

In [23]:
train_nonspam_bow=dict()
for w in common_words:
    train_nonspam_bow[w]=train_nonspam_words.count(w)/len(train_nonspam_words)

Predict on Test Set
P(SPAM|"you have won as a valued vodafone customer")∝ P("you have won as a valued vodafone customer"|SPAM)*P(SPAM)

 we make IID assumption on the words so it can be written as

P("You"|SPAM).P("have"|SPAM)...................P("customer"|SPAM)*P(SPAM)

I=ea

In [17]:
def predict(t,verbose=False):
    valid_words=[w for w in t if w in train_spam_bow]
    spam_prob=[train_spam_bow[w] for w in valid_words]
    nonspam_prob=[train_nonspam_bow[w] for w in valid_words]
    
    if verbose:
        df=pd.DataFrame()
        df['words']=valid_words
        df['spam_prob']=spam_prob
        df['nonspam_prob']=nonspam_prob
        df['ratio']=[s/n if n>0 else np.inf for s,n in zip(spam_prob,nonspam_prob)]
        print(df)
    
    spam_score=sum([np.log(p) for p in spam_prob])+np.log(fraction_spam)
    
    nonspam_score=sum([np.log(p) for p in nonspam_prob])+(1-np.log(fraction_spam))
    
    if verbose:
        print("spam score is :",spam_score)
        print("NOn spam score is :",nonspam_score)
    return spam_score>=nonspam_score

In [18]:
predict("urgent call this number".split(' '),verbose=True)

    words  spam_prob  nonspam_prob      ratio
0  urgent   0.000753      0.000022  33.998075
1    call   0.019422      0.003302   5.881844
2    this   0.005274      0.003590   1.469053
3  number   0.001758      0.000643   2.735477
spam score is : -24.741342083531464
NOn spam score is : -26.389062064519663


True

In [19]:
predict("hey do you want to go to rtocinema".split(' '),verbose=True)

  words  spam_prob  nonspam_prob     ratio
0   hey   0.000167      0.000377  0.444419
1    do   0.000419      0.005430  0.077093
2   you   0.014064      0.026660  0.527540
3  want   0.001591      0.002393  0.664571
4    to   0.040603      0.024289  1.671641
5    go   0.002093      0.003546  0.590244
6    to   0.040603      0.024289  1.671641
spam score is : -41.77900371762176
NOn spam score is : -32.816147407704364


False

In [20]:
predictions=testing_df.text.apply(lambda t: predict(t.split()))

In [21]:
recall=np.sum((predictions==True) & testing_df.spam==True)/np.sum(testing_df.spam==True)
print('Fraction Spam Correctly Detected: %s'%recall)


Fraction Spam Correctly Detected: 0.8391304347826087


In [22]:
fraction_of_valid_messages_sent_to_spam = np.sum((predictions == True) & (testing_df.spam == False)) / np.sum(testing_df.spam == False)
print('Fraction Valid Messages Sent to Spam: %s'%fraction_of_valid_messages_sent_to_spam)


Fraction Valid Messages Sent to Spam: 0.004160887656033287
